Model wird trainiert mit einem verkleinerten Feature set trainiert um den Anforderungen des neu gezogenen Datensatzes zu genügen.

In [4]:
import pandas as pd
import pickle
from datetime import datetime
%pylab inline

dat_all_wBehav, dat_long_wBehav, dat_short_wBehav = pickle.load(open('dat_to_train_wlastBehav_reducedFeat.pkl','rb'))

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

# prepare data
keys = ['dat_all_wBehav', 'dat_long_wBehav', 'dat_short_wBehav']
dats = [dat_all_wBehav, dat_long_wBehav, dat_short_wBehav]

dat_train = {}
dat_test = {}
label_train = {}
label_test = {}
for idx, key in enumerate(keys):
    label = dats[idx]['dat_abs_gauss_final']['Kuendigungsstatus']
    dat_cp = dats[idx]['dat_abs_gauss_final'].drop(columns = ['Kuendigungsstatus','Newsletter'])
    
    dat_train_, dat_test_, label_train_, label_test_ = train_test_split(dat_cp, label, test_size=0.2, random_state=42)
    dat_train[key] = dat_train_
    dat_test[key] = dat_test_
    label_train[key] = label_train_
    label_test[key] = label_test_

In [19]:
# model training
classifiers = {}
for key in keys:
    
    print(datetime.now().time())
    print(key)

    classifiers[key] = GradientBoostingClassifier( n_estimators = 800 )
    classifiers[key].fit(dat_train[key], label_train[key])

18:44:28.806563
dat_all_wBehav
18:44:46.437242
dat_long_wBehav
18:44:56.025785
dat_short_wBehav


In [20]:
## save model
import pickle
pickle.dump( classifiers['dat_all_wBehav'], open( "real_prediction/trainedModel.pkl", "wb" ) )

In [14]:
from sklearn.metrics import classification_report

pred_train={}
pred_test ={}

for key in keys:
    pred_train[key] = classifiers[key].predict(dat_train[key])
    pred_test[key] = classifiers[key].predict(dat_test[key])
    


In [15]:
print('#### Training with n_estimators = 800 ###')

for key in keys:
    print('#########################')
    print(key)
    print('#########################')
    print('TRAINING SET')
    print('')
    print(pd.crosstab(label_train[key], pred_train[key], rownames=['True'], colnames=['Predicted'], margins=True))
    print()
    print(classification_report(label_train[key], pred_train[key]))
    print('#########################')
    print('TEST SET')
    print('')
    print(pd.crosstab(label_test[key], pred_test[key], rownames=['True'], colnames=['Predicted'], margins=True))
    print()
    print(classification_report(label_test[key], pred_test[key]))

#### Training with n_estimators = 800 ###
#########################
dat_all_wBehav
#########################
TRAINING SET

Predicted     gekuendigt  ungekuendigt   All
True                                        
gekuendigt          1693            17  1710
ungekuendigt           6          5108  5114
All                 1699          5125  6824

              precision    recall  f1-score   support

  gekuendigt       1.00      0.99      0.99      1710
ungekuendigt       1.00      1.00      1.00      5114

 avg / total       1.00      1.00      1.00      6824

#########################
TEST SET

Predicted     gekuendigt  ungekuendigt   All
True                                        
gekuendigt           408            27   435
ungekuendigt           2          1269  1271
All                  410          1296  1706

              precision    recall  f1-score   support

  gekuendigt       1.00      0.94      0.97       435
ungekuendigt       0.98      1.00      0.99      1271

 avg /